In [1]:
import serial
# from ReadLine import read_safe, send_message, clear_output, interpret
from func_timeout import func_timeout, FunctionTimedOut
import numpy as np
import ast
from matplotlib import pyplot as plt
import time
import json

In [2]:
ser = serial.Serial()  # open serial port
ser.port="/dev/tty.usbmodem101"
ser.baudrate=250000
print(ser)

Serial<id=0x112cf0100, open=False>(port='/dev/tty.usbmodem101', baudrate=250000, bytesize=8, parity='N', stopbits=1, timeout=None, xonxoff=False, rtscts=False, dsrdtr=False)


In [5]:
ser.is_open

True

In [4]:
ser.open()

The Comms Class (implementation of the Readline class) 

In [6]:
class CommsReadline: 
    def __init__ (self, serial_conn, divider=":"):
        self.ser = serial_conn
        self.divider = divider

    def input_timeout(self, timeout, *args):
        try:
            return func_timeout(timeout, input, args=args)
        except FunctionTimedOut:
            pass
    def read_safe(self, timeoutval = 0.1):
        try:
            line = func_timeout(timeoutval, self.ser.readline)
            # print(line)
            return(line)
        except FunctionTimedOut:
            # print("Exception: No line to read")
            return(None)
    
    def send_message(self, message):
        self.ser.write((message + "\n").encode())
        time.sleep(0.2)
    
    def clear_output(self):
        data = 1
        while data is not None:
            data = self.read_safe()
    
    def interpret(self, byteval):
        if byteval is None:
            return
        strval = byteval.decode("utf-8")
        # print(strval, "printing the strval")
        messages = strval.split("\r")[0].split(self.divider)
        # print(messages)
        try:
            msgtype = messages[1]
            if msgtype == 'msg':
                msg = messages[2]
                # print(msg)
                return msg, None
            elif msgtype == 'msgvar':
                varname = messages[2]
                val = messages[3]
                print("{}, {}".format(varname,val))
                return varname, val
            elif msgtype == '1Darr':
                varname = messages[2]
                array = ast.literal_eval(messages[3])
                # print("{}, {}".format(varname,len(array)))
                return varname, array
            else:
                return None, None
        except: 
            print("exception:")
            print(messages)
            return (None, None)

In [40]:
# def clear_output(ser):
#     data = 1
#     while data is not None:
#         data = read_safe(ser)

In [41]:
# def read_safe(ser, timeoutval = 0.1):
#     try:
#         line = func_timeout(timeoutval, ser.readline)
#         # print(line)
#         return(line)
#     except FunctionTimedOut:
#         # print("Exception: No line to read")
#         return(None)

In [42]:
# def send_message(ser, message, divider=":"):
#     ser.write((message + "\n").encode())
#     time.sleep(0.2)

In [44]:
# send_message(ser, "print;")

In [47]:
# clear_output(ser)
# read_safe(ser)

Instantiating the comms object

In [7]:
comms = CommsReadline(ser)

In [7]:
comms.send_message("print;")

In [8]:
# comms.send_message("addparam;FBshift;baseline;offset;width;")
# comms.send_message("print;")
# data=1
# while data is not None:
#     data = comms.read_safe()

In [9]:
# comms.send_message("print;")
# data=1
# while data is not None:
#     data = comms.read_safe()

In [10]:
# comms.send_message("print;")
# data = comms.read_safe()
# comms.interpret(data)

['    ', 'msgvar', 'FBshift', '0', '    ']
FBshift, 0


('FBshift', '0')

In [10]:
# comms.clear_output()

In [8]:
# comms.send_message("addparam;FBshift;baseline;offset;width;")

In [23]:
# comms.send_message("resetparam;")

In [24]:
# comms.send_message("addparam;FBshift;baseline;offset;width;")

In [8]:
# comms.send_message("resetparam;")
comms.send_message("print;")
data = 1

initializeDict = False
listODicts = []
while data is not None:
    data = comms.read_safe()
    # print(data)
    if data is None:
        break
    varname, val = comms.interpret(data)

    if varname == "finished":
        break

    if varname == "finalize":
        initializeDict = False
        listODicts.append(placeholderDict)
    
    if initializeDict:
        placeholderDict[varname] = val
    
    if varname == "initialize":
        initializeDict = True
        placeholderDict = {}

FBshift, 0
baseline, 0
offset, 0
width, 0


In [10]:
comms.clear_output()

In [11]:
comms.send_message("Z")
data = 1

initializeDict = False
listODicts = []
while data is not None:
    data = comms.read_safe()
    print(data)

None


In [ ]:
comms.clear_output()

comms.send_message("X")
data = 1

initializeDict = False
listODicts = []
while data is not None:
    data = comms.read_safe(timeoutval = 1000.)
    if data is None:
        break
    varname, val = comms.interpret(data)

    if varname == "finished":
        break

    if varname == "finalize":
        initializeDict = False
        listODicts.append(placeholderDict)
    
    if initializeDict:
        placeholderDict[varname] = val
    
    if varname == "initialize":
        initializeDict = True
        placeholderDict = {}
print(listODicts)

In [ ]:
len(listODicts)

In [ ]:
for key, val in listODicts[0].items():
    print(key)
    if isinstance(val, list):
        print(len(val))

In [10]:
ser.close()

In [77]:
with open('data/biastest_all_FCswitched.json', 'w') as f:
     json.dump(listODicts, f)